# *Lab 2 - Netflix Recommendation System*

In [2]:
import pandas as pd
import numpy as np

In [4]:
cData = pd.read_csv('credits.csv')
cData.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [395]:
m2Data = pd.read_csv('movies_metadata_2.csv', low_memory=False)
m2Data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [396]:
m2Data.release_date = pd.to_datetime(m2Data.release_date, dayfirst=False,errors = 'coerce')

In [397]:
m2Data['Year'] = m2Data.release_date.dt.year

In [398]:
m2Data['Year'].value_counts().sort_index()

1874.0       1
1878.0       1
1883.0       1
1887.0       1
1888.0       2
          ... 
2015.0    1905
2016.0    1604
2017.0     532
2018.0       5
2020.0       1
Name: Year, Length: 135, dtype: int64

In [399]:
# data upto 2020 but we will take till 2017 as there is not much in 2018, 2019,2020
# We already have data for years upto 2016 so we will take for 2017 for this dataset

In [400]:
new_meta_data = m2Data[['genres','id','title','Year']].copy()

In [401]:
new_meta_data = new_meta_data[new_meta_data['Year'] == 2017]

In [402]:
new_meta_data

,genres,id,title,Year
26560,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",166426,Pirates of the Caribbean: Dead Men Tell No Tales,2017.0
26561,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",141052,Justice League,2017.0
26565,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",284053,Thor: Ragnarok,2017.0
26566,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",283995,Guardians of the Galaxy Vol. 2,2017.0
30536,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",245842,The King's Daughter,2017.0
...,...,...,...,...
45398,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",468707,Thick Lashes of Lauri Mäntyvaara,2017.0
45417,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",461297,Cop and a Half: New Recruit,2017.0
45437,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",455661,In a Heartbeat,2017.0
45453,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",404604,Mom,2017.0


In [403]:
cData.dtypes

cast    object
crew    object
id       int64
dtype: object

In [404]:
new_meta_data['id'] = new_meta_data['id'].astype('int64')

In [405]:
new_meta_data.dtypes

genres     object
id          int64
title      object
Year      float64
dtype: object

In [406]:
data = pd.merge(cData,new_meta_data, on ='id')

In [407]:
import ast

In [408]:
data['genres'] = data['genres'].map(lambda x:ast.literal_eval(x))
data['cast'] = data['cast'].map(lambda x:ast.literal_eval(x))
data['crew'] = data['crew'].map(lambda x:ast.literal_eval(x))

In [409]:
#Now we will have to take out the individual genres like adventure, action, sci-fi, etc. using the following function


def make_genresList(x):
    gen = []
    st = " "
    for i in x:
        if i.get('name') == 'Science Fiction': #am only renaming the "Science Fiction" to "Sci-Fi" to make it the name shorter. Apart from that all other names remain the same.
            scifi = 'Sci-Fi'
            gen.append(scifi)
        else:
            gen.append(i.get('name'))
    if gen == []:
        return np.NaN
    else:
        return (st.join(gen)) #then we will join them together and return the valuse

In [413]:
data['genres'] = data['genres'].map(make_genresList)

In [414]:

#let's do the same for cast

def get_actor1(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == []:
        return np.NaN
    else:
        return (casts[0])

In [418]:
data['actor_1_name'] = data['cast'].map(get_actor1)

In [416]:

def get_actor2(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=1:
        return np.NaN
    else:
        return (casts[1])

In [425]:
data['actor_2_name'] = data['cast'].map(get_actor2)

In [426]:

def get_actor3(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=2:
        return np.NaN
    else:
        return (casts[2])

In [427]:
data['actor_3_name'] = data['cast'].map(get_actor3)

In [438]:
def get_directors(x):
    dt = []
    st = " "
    for i in x:
        if i.get('job') == 'Director':
            dt.append(i.get('name'))
    if dt == []:
        return np.NaN
    else:
        return (st.join(dt))

In [439]:
data['director_name'] = data['crew'].map(get_directors)

In [440]:
movie = data.loc[:,['title','actor_1_name','actor_2_name','actor_3_name','director_name','genres']].copy()

In [441]:
movie

,title,actor_1_name,actor_2_name,actor_3_name,director_name,genres
0,Pirates of the Caribbean: Dead Men Tell No Tales,Johnny Depp,Javier Bardem,Geoffrey Rush,Joachim Rønning Espen Sandberg,Adventure Action Fantasy Comedy
1,Justice League,Ben Affleck,Henry Cavill,Gal Gadot,Zack Snyder,Action Adventure Fantasy Sci-Fi
2,Thor: Ragnarok,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Taika Waititi,Action Adventure Fantasy Sci-Fi
3,Guardians of the Galaxy Vol. 2,Chris Pratt,Zoe Saldana,Dave Bautista,James Gunn,Action Adventure Comedy Sci-Fi
4,The King's Daughter,Pierce Brosnan,William Hurt,Benjamin Walker,Sean McNamara,Fantasy Action Adventure
...,...,...,...,...,...,...
526,Thick Lashes of Lauri Mäntyvaara,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Hannaleena Hauru,Romance Comedy
527,Cop and a Half: New Recruit,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Jonathan A. Rosenbaum,Crime Comedy Action Family
528,In a Heartbeat,NaN,NaN,NaN,Beth David Esteban Bravo,Family Animation Romance Comedy
529,Mom,Sridevi Kapoor,Sajal Ali,Akshaye Khanna,Ravi Udyawar,Crime Drama Thriller


In [442]:
movie.isna().sum()

title             0
actor_1_name     22
actor_2_name     55
actor_3_name     70
director_name     4
genres            7
dtype: int64

In [443]:
# Dropping missing columns

In [444]:
movie.dropna(how='any', inplace=True)

In [445]:
movie.isna().sum()

title            0
actor_1_name     0
actor_2_name     0
actor_3_name     0
director_name    0
genres           0
dtype: int64

In [451]:
movie.rename(columns={'title':'movie_title'}, inplace=True)

In [452]:
movie

,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,genres
0,Pirates of the Caribbean: Dead Men Tell No Tales,Johnny Depp,Javier Bardem,Geoffrey Rush,Joachim Rønning Espen Sandberg,Adventure Action Fantasy Comedy
1,Justice League,Ben Affleck,Henry Cavill,Gal Gadot,Zack Snyder,Action Adventure Fantasy Sci-Fi
2,Thor: Ragnarok,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Taika Waititi,Action Adventure Fantasy Sci-Fi
3,Guardians of the Galaxy Vol. 2,Chris Pratt,Zoe Saldana,Dave Bautista,James Gunn,Action Adventure Comedy Sci-Fi
4,The King's Daughter,Pierce Brosnan,William Hurt,Benjamin Walker,Sean McNamara,Fantasy Action Adventure
...,...,...,...,...,...,...
524,The Incredible Jessica James,Jessica Williams,Chris O'Dowd,Keith Stanfield,Jim Strouse,Romance Comedy
525,Can't Buy My Love,Adelaide Kane,Benjamin Hollingsworth,Jean Louisa Kelly,Farhad Mann,Romance
526,Thick Lashes of Lauri Mäntyvaara,Inka Haapamäki,Rosa Honkonen,Tiitus Rantala,Hannaleena Hauru,Romance Comedy
527,Cop and a Half: New Recruit,Lou Diamond Phillips,Wallace Shawn,Gina Holden,Jonathan A. Rosenbaum,Crime Comedy Action Family


In [454]:
movie.movie_title = movie.movie_title.str.lower()

In [455]:
movie.columns

Index(['movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name',
       'director_name', 'genres'],
      dtype='object')

In [458]:
movie['Combined'] = movie['movie_title'] + movie['actor_1_name']+ movie['actor_2_name']+ movie['actor_3_name']+movie['director_name']+ movie['genres']

In [460]:
movie.to_csv('Processed/PrimaryData2.csv')

In [461]:
# Loading old dataset

In [467]:
temp = pd.read_csv('Processed/PrimaryData.csv',index_col = 'Unnamed: 0')
temp

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...
...,...,...,...,...,...,...
5038,Scott Smith,Eric Mabius,Daphne Zuniga,Crystal Lowe,Comedy Drama,signed sealed delivered
5039,unknown,Natalie Zea,Valorie Curry,Sam Underwood,Crime Drama Mystery Thriller,the following
5040,Benjamin Roberds,Eva Boehnke,Maxwell Moody,David Chandler,Drama Horror Thriller,a plague so pleasant
5041,Daniel Hsia,Alan Ruck,Daniel Henney,Eliza Coupe,Comedy Drama Romance,shanghai calling


In [468]:
temp['Combined'] = temp['movie_title'] + temp['actor_1_name']+ temp['actor_2_name']+ temp['actor_3_name']+temp['director_name']+ temp['genres']

In [469]:
temp

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,Combined
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,avatarCCH PounderJoel David MooreWes StudiJame...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,pirates of the caribbean: at world's endJohnny...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,spectreChristoph WaltzRory KinnearStephanie Si...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,the dark knight risesTom HardyChristian BaleJo...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,star wars: episode vii - the force awakens ...
...,...,...,...,...,...,...,...
5038,Scott Smith,Eric Mabius,Daphne Zuniga,Crystal Lowe,Comedy Drama,signed sealed delivered,signed sealed deliveredEric MabiusDaphne Zunig...
5039,unknown,Natalie Zea,Valorie Curry,Sam Underwood,Crime Drama Mystery Thriller,the following,the following Natalie ZeaValorie Cu...
5040,Benjamin Roberds,Eva Boehnke,Maxwell Moody,David Chandler,Drama Horror Thriller,a plague so pleasant,a plague so pleasantEva BoehnkeMaxwell MoodyDa...
5041,Daniel Hsia,Alan Ruck,Daniel Henney,Eliza Coupe,Comedy Drama Romance,shanghai calling,shanghai callingAlan RuckDaniel HenneyEliza Co...


In [477]:
final = pd.concat([movie, temp], axis=0)

In [478]:
final.reset_index(inplace=True)

In [479]:
final.drop_duplicates(subset='movie_title', keep='last', inplace=True)

In [480]:
final.shape

(5364, 8)

In [481]:
final.drop()

,index,movie_title,actor_1_name,actor_2_name,actor_3_name,director_name,genres,Combined
0,0,pirates of the caribbean: dead men tell no tales,Johnny Depp,Javier Bardem,Geoffrey Rush,Joachim Rønning Espen Sandberg,Adventure Action Fantasy Comedy,pirates of the caribbean: dead men tell no tal...
1,1,justice league,Ben Affleck,Henry Cavill,Gal Gadot,Zack Snyder,Action Adventure Fantasy Sci-Fi,justice leagueBen AffleckHenry CavillGal Gadot...
2,2,thor: ragnarok,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Taika Waititi,Action Adventure Fantasy Sci-Fi,thor: ragnarokChris HemsworthTom HiddlestonCat...
3,3,guardians of the galaxy vol. 2,Chris Pratt,Zoe Saldana,Dave Bautista,James Gunn,Action Adventure Comedy Sci-Fi,guardians of the galaxy vol. 2Chris PrattZoe S...
4,4,the king's daughter,Pierce Brosnan,William Hurt,Benjamin Walker,Sean McNamara,Fantasy Action Adventure,the king's daughterPierce BrosnanWilliam HurtB...
...,...,...,...,...,...,...,...,...
5496,5038,signed sealed delivered,Eric Mabius,Daphne Zuniga,Crystal Lowe,Scott Smith,Comedy Drama,signed sealed deliveredEric MabiusDaphne Zunig...
5497,5039,the following,Natalie Zea,Valorie Curry,Sam Underwood,unknown,Crime Drama Mystery Thriller,the following Natalie ZeaValorie Cu...
5498,5040,a plague so pleasant,Eva Boehnke,Maxwell Moody,David Chandler,Benjamin Roberds,Drama Horror Thriller,a plague so pleasantEva BoehnkeMaxwell MoodyDa...
5499,5041,shanghai calling,Alan Ruck,Daniel Henney,Eliza Coupe,Daniel Hsia,Comedy Drama Romance,shanghai callingAlan RuckDaniel HenneyEliza Co...
